<a href="https://colab.research.google.com/github/online6731/EDF-Scheduling/blob/main/EDF_Scheduling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

## Problem

Write an scheduler program for real-time systems based on one of the following algorithms:
- Monotonic Rate
- Earliest Deadline First
- Least Laxity

# Environment

In [ ]:
#@title Install { vertical-output: true }
 
# NOTE: Restart the runtime after installation
! pip install plotly==4.9

     |████████████████████████████████| 12.9MB 251kB/s 
  Found existing installation: plotly 4.4.1
    Uninstalling plotly-4.4.1:
      Successfully uninstalled plotly-4.4.1


In [ ]:
#@title Import { vertical-output: true }
 
from google.colab.data_table import DataTable
import plotly.express as px
import pandas as pd
import numpy as np
import json

In [ ]:
#@title Modules { vertical-output: true }
 
def to_date(i):
    'returns date for number to use in timeline charts'
 
    return f'2000-01-{i+1}' if type(i) == int else i

# Solution

In [ ]:
#@title Tasks { vertical-output: true }
 
%%writefile tasks.json
{
    "Task_1": {"capacity": 3, "deadline": 7, "period": 20},
    "Task_2": {"capacity": 2, "deadline": 4, "period": 5},
    "Task_3": {"capacity": 2, "deadline": 8, "period": 10}
}

Writing tasks.json


In [ ]:
#@title Scheduler { vertical-output: true }
 
def schedule(tasks):
    tasks = pd.DataFrame.from_dict(tasks, orient='index')
 
    tasks['current_capacity'] = tasks['capacity']
    tasks['current_deadline'] = tasks['deadline']
    timeline = []
 
    # build the timeline for the required time slots 
    for i in range(np.lcm.reduce(tasks.period)):
        
        # filter the completed tasks
        left_tasks = tasks[tasks['current_capacity'] > 0]
 
        if len(left_tasks) > 0:
            # find the task with the closest deadline
            top_task = left_tasks.sort_values('current_deadline').index[0]
            
            # decrease the capacity of current task by one
            tasks.loc[top_task]['current_capacity'] -= 1
            
            if 0 < i and timeline[-1]['task'] == top_task and timeline[-1]['end'] == i:
                # if current task is the same as the last task, update it's end time and length
                timeline[-1].update({'end': i+1,
                                     'length': timeline[-1]['length'] + 1})
            else:
                # otherwise add a the new task to the timeline
                timeline += [{'task': top_task,
                              'start': i,
                              'end': i+1, 
                              'length': 1}]
        
        # update the capacity and the deadline of newly arrived tasks
        arrived = tasks[(i + 1) % tasks['period'] == 0].index
        tasks.loc[arrived, 'current_capacity'] = tasks.loc[arrived, 'capacity']
        tasks.loc[arrived, 'current_deadline'] = tasks.loc[arrived, 'deadline'] + i + 2
 
    return pd.DataFrame(timeline)
 
timeline = schedule(json.load(open('tasks.json')))

DataTable(timeline)

,task,start,end,length
0,Task_2,0,2,2
1,Task_1,2,5,3
2,Task_3,5,7,2
3,Task_2,7,9,2
4,Task_2,10,12,2
5,Task_3,12,14,2
6,Task_2,15,17,2


In [ ]:
#@title Timeline Chart
 
# update the start and end to dates
timeline['start'] = timeline['start'].apply(to_date)
timeline['end'] = timeline['end'].apply(to_date)
 
# create the timeline chart
fig = px.timeline(timeline, 
                  x_start="start",
                  x_end="end", 
                  y="task",
                  color="task")
 
# fix the x axis to show numbers instead of dates
fig.layout.xaxis.tickformat = '%j'
 
fig